In [23]:
import pandas as pd
import numpy as np
from model import *
# import tensorflow as tf
# import keras
# from keras.models import Model
# from keras.layers import Input, Dense, ReLU
# from keras import optimizers, layers
# from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [24]:
import statistics

rain = pd.read_csv("./Data/Rainfall.csv")

lpa = statistics.mean(list(rain['Jun-Sep'][12:41]))
rain_fall_data = list(rain['Jun-Sep'])

rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

In [25]:
import glob

slp_path  = "./Data/Anomaly/slp/*"
sst_path  = "./Data/Anomaly/sst/*"
uwnd_path = "./Data/Anomaly/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

In [26]:
s = 0
e = 52

In [27]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd = slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd = pd.read_csv(month_path_slp, header=[0, 1], index_col=0), pd.read_csv(month_path_sst, header=[0, 1], index_col=0), pd.read_csv(month_path_uwnd, header=[0, 1], index_col=0)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[s:e], feature_sst[s:e], feature_uwnd[s:e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)

combine = [np.array(combine_slp), np.array(combine_sst), np.array(combine_uwnd)]
month_data_slp.head()

Latitude         -70                                                         \
Longitude        180        200        220        240        260        280   
1949       -3.649323  -3.649323  -3.649323  -3.649323  -3.649323  -3.649323   
1950        1.230677   1.230677   1.230677   1.230677   1.230677   1.230677   
1951        7.090677   7.090677   7.090677   7.090677   7.090677   7.090677   
1952        8.300677   8.300677   8.300677   8.300677   8.300677   8.300677   
1953       10.240677  10.240677  10.240677  10.240677  10.240677  10.240677   

Latitude                               -60             ...  \
Longitude        320        340          0         20  ...   
1949       -3.649323  -3.649323  -3.649323  -3.649323  ...   
1950        1.230677   1.230677   1.230677   1.230677  ...   
1951        7.090677   7.090677   7.090677   7.090677  ...   
1952        8.300677   8.300677   8.300677   8.300677  ...   
1953       10.240677  10.240677  10.240677  10.240677  ...   

Latitude  Unnamed: 315_level_0 Unnamed: 316_level_0 Unnamed: 317_level_0  \
Longitude Unnamed: 315_level_1 Unnamed: 316_level_1 Unnamed: 317_level_1   
1949                  6.598676             5.294682             3.563006   
1950                 -4.251374            -4.835348            -5.186994   
1951                  0.478676             3.114682             5.042976   
1952                  8.528676             9.234682             9.053006   
1953                 -1.301354            -1.155358            -1.876994   

Latitude  Unnamed: 318_level_0 Unnamed: 319_level_0 Unnamed: 320_level_0  \
Longitude Unnamed: 318_level_1 Unnamed: 319_level_1 Unnamed: 320_level_1   
1949                  1.995687             1.005682             0.951014   
1950                 -4.854313            -2.824318            -0.829026   
1951                  5.235657             5.285642             5.451014   
1952                  7.765687             5.885682             4.100964   
1953                 -3.864313            -5.174318            -4.628986   

Latitude  Unnamed: 321_level_0 Unnamed: 322_level_0 Unnamed: 323_level_0  \
Longitude Unnamed: 321_level_1 Unnamed: 322_level_1 Unnamed: 323_level_1   
1949                  1.386347              1.43035             1.857351   
1950                 -1.153683             -1.92965            -1.202699   
1951                  5.246347              5.95035             6.487301   
1952                  2.876347              2.64035             3.047301   
1953                 -2.843693             -2.32968            -3.162649   

Latitude  Unnamed: 324_level_0  
Longitude Unnamed: 324_level_1  
1949                  3.877972  
1950                  0.617962  
1951                  4.937972  
1952                  2.927962  
1953                 -2.762038  

[5 rows x 324 columns]

In [28]:
def get_data(List):
    N = len(List)
    if N == 1:
        return combine[List[0]]
    elif N == 2:
        return np.concatenate((combine[List[0]], combine[List[1]]), axis=1)
    else:
        return np.concatenate((combine[List[0]], combine[List[1]], combine[List[2]]), axis=1)


In [29]:
inputs = get_data([0])
inputs.shape
models = get_trained_model([324, 79, 29, 9], inputs)

Epoch 1/800


ValueError: in user code:

    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Sahaj\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la